# Import and init

In [8]:
import os 
import json

import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk 
from nltk.corpus import stopwords

import config

# Read data

In [9]:
df0 = pd.read_csv(config.INPUT_DATA_RELPATH)
df0.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


# Data preproc

## Prepared data (`df1`)

In [10]:
df1 = df0.copy(deep = True)

# Fill null values
df1 = (
    df1[
        ~df1['overview'].isnull()
    ]
    .reset_index(drop = True)
)

df1

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4796,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4797,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

## Ingest to model (`df2`)

In [11]:
df2 = df1.copy(deep = True)
# Filter columns
# df2 = df2[['overview']]
df2 = df2['overview'].tolist()
print(len(df2))
df2[:5]

4800


['In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.',
 'Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.',
 'A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE.',
 "Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a

# Text vectorisation (TF-IDF)

In [12]:
vectorizer = TfidfVectorizer(
    strip_accents = 'unicode',
    lowercase = True, 
    stop_words = stopwords.words('english'),
    ngram_range = (1, 2),
    min_df = 2,
    # max_features = 40
)
tfidf_matrix = vectorizer.fit_transform(
    df2
)
vectorizer.get_feature_names_out()

array(['00', '00 agent', '000', ..., 'zookeeper', 'zoologists', 'zorro'],
      shape=(18898,), dtype=object)

In [13]:
tfidf_matrix_dense = tfidf_matrix.toarray()
tfidf_matrix_dense[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(5, 18898))

# Model recommendation

In [14]:
def get_index_of_movie_title(df: pd.DataFrame, 
                             title: str,
                             column: str = 'title'
                             ) -> int:
    """
    Based on the movie title, get its index in the corpus dataframe.
    If multiple matches are found, get the first one.

    Example usage:
    ```py
    get_index_of_movie_title(df1, which_word)
    >>> 7
    ```
    """
    return int(
        df[
            df[column] == title
        ]
        .index[0]
    )



In [31]:
def get_top_n_similar_indices(title: str,
                              df: pd.dataFrame
                              ) -> None:
    """
    Based on an input of movie title (that has to be present in the training corpus),
    produce n = 5 recommendations movies. 
    """
    # Get index of the movie title
    a = get_index_of_movie_title(df, title)

    # Get similarity matrix of this movie's description to other descriptions
    cosine_similarity_matrix = []
    for i in tfidf_matrix_dense:
        idx = cosine_similarity([i], [tfidf_matrix_dense[a]])[0][0]
        cosine_similarity_matrix.append(float(idx))
    
    # Get top 5 indices with the highest similarity score
    ind = np.argpartition(cosine_similarity_matrix, -5)[-5:]
    indices, cosine_similarity_scores = [], []
    for i in ind[::-1]:
        indices.append(i)
        cosine_similarity_scores.append(cosine_similarity_matrix[i])

    return indices, cosine_similarity_scores

def format_recommendations(title: str,
                           training_data: pd.DataFrame,
                           indices_in_training_data: int,
                           cosine_similarity_scores: float,
                           format_print: str = 'list'
                           ) -> None:
    """
    Args:
        format_print: can be 'list', 'table'
    """
    if format_print == 'list':
        # Print the recommendations
        print(f"Top 5 words similar to '{title}':\n")
        for index, i in enumerate(indices_in_training_data):
            print(f"- index: {i}")
            print(f"- cosine similarity score: {cosine_similarity_scores[index]}")
            print(f" - recommended title: '{training_data.iloc[i]['title']}'")
            print(f" - tagline: {training_data.iloc[i]['tagline']}")
            print('\n')
    elif format_print == 'table':
        indices, cosine, title, tagline = [], [], [], []
        for index, i in enumerate(indices_in_training_data):
            indices.append(i)
            cosine.append(
                cosine_similarity_scores[index]
            )
            title.append(
                training_data.iloc[i]['title']
            )
            tagline.append(
                training_data.iloc[i]['tagline']
            )
        df_markdown = pd.DataFrame({
            'index': indices, 
            'cosine': cosine,
            'title': title, 
            'tagline': tagline
        })
        print(df_markdown.to_markdown(index = False))
        pass
    return None

def get_n_recommendations(title: str,
                          print_format: str = 'list'
                          ) -> None:
    ind, cosine_similarity_scores = get_top_n_similar_indices(title, df1)
    print(f'Recommendations for the movie "{title}":\n')
    format_recommendations(title, df1, ind, cosine_similarity_scores, print_format)
    print('\n')
    return None

get_n_recommendations('The Avengers', print_format = 'table')

Recommendations for the movie "The Avengers":

|   index |   cosine | title                           | tagline                                      |
|--------:|---------:|:--------------------------------|:---------------------------------------------|
|      16 | 1        | The Avengers                    | Some assembly required.                      |
|      91 | 0.153163 | Independence Day: Resurgence    | We had twenty years to prepare. So did they. |
|     256 | 0.12732  | Allegiant                       | Break the boundaries of your world           |
|       7 | 0.115689 | Avengers: Age of Ultron         | A New Age Has Come.                          |
|     588 | 0.109449 | Wall Street: Money Never Sleeps | Gordon never gives up.                       |




In [32]:
for i in (
    'Zombieland', 
    '28 Days Later'
):
    get_n_recommendations(i, 'table')


Recommendations for the movie "Zombieland":

|   index |   cosine | title            | tagline                                             |
|--------:|---------:|:-----------------|:----------------------------------------------------|
|    1988 | 1        | Zombieland       | This place is so dead                               |
|    2308 | 0.187876 | Land of the Dead | The dead shall inherit the Earth.                   |
|    1740 | 0.137952 | Kick-Ass 2       | You Can't Fight Your Destiny.                       |
|    2387 | 0.113387 | Day of the Dead  | The darkest day of horror the world has ever known. |
|    3559 | 0.109904 | Ouija            | Keep telling yourself it's just a game              |


Recommendations for the movie "28 Days Later":

|   index |   cosine | title         | tagline                                                                            |
|--------:|---------:|:--------------|:----------------------------------------------------------------------